https://adventofcode.com/2022/day/14

In [1]:
import operator
import itertools as it
from copy import deepcopy
from enum import Enum
from typing import Optional

import numpy as np


class Field(Enum):
    AIR = 0
    ROCK = 1
    SAND = 2


Sand = tuple[int, int]


def parseField(input_file: str) -> np.array:
    with open(input_file) as f:
        text = f.read()

    rockLines = list()
    dimx, dimy = 0, 0
    for l in text.splitlines():
        line = [tuple(map(lambda x: int(x.strip()), sxy.split(","))) for sxy in l.split("->")]
        rockLines.append(line)
        dimx, dimy = max([dimx] + [x for x, _ in line]), max([dimy] + [y for _, y in line])

    field = Field.AIR.value * np.ones((dimy + 1, dimx + 1), dtype=int)
    for rl in rockLines:
        for i in range(len(rl) - 1):
            start, end = map(np.array, (rl[i], rl[i + 1]))
            p = deepcopy(start)
            dx = 1 if start[0] < end[0] else -1 if start[0] > end[0] else 0
            dy = 1 if start[1] < end[1] else -1 if start[1] > end[1] else 0
            while not all(p == end):
                field[p[1], p[0]] = Field.ROCK.value
                p += (dx, dy)
            field[p[1], p[0]] = Field.ROCK.value

    return field


def updateField(grain: Sand, field: np.array) -> Optional[Sand]:
    if field[grain[1], grain[0]] != Field.AIR.value:
        result = grain
    elif field[grain[1] + 1, grain[0]] == Field.AIR.value:
        result = Sand((grain[0], grain[1] + 1))
    else:
        if field[grain[1] + 1, grain[0] - 1] == Field.AIR.value:
            result = Sand((grain[0] - 1, grain[1] + 1))
        elif field[grain[1] + 1, grain[0] + 1] == Field.AIR.value:
            result = Sand((grain[0] + 1, grain[1] + 1))
        else:
            result = None
            field[grain[1], grain[0]] = Field.SAND.value

    return result


def solvePart1(field: np.array) -> int:
    SEED = (500, 0)
    g = None
    while True:
        if g is None:
            g = Sand(SEED)
        try:
            g = updateField(g, field)
        except IndexError:
            break
    return sum((field == Field.SAND.value).flatten())


def solvePart2(field: np.array) -> int:
    SEED = (500, 0)
    for i in range(field.shape[0] - 1, -1, -1):
        if any(field[i,:] != Field.AIR.value):
            h = i + 2
            break
    field = np.pad(field, ((0,2),(0,0)))
    field[h] = Field.ROCK.value

    g = None
    while True:
        if g is None:
            g = Sand(SEED)

        g = updateField(g, field)

        if g == SEED:
            field[SEED[1], SEED[0]] = Field.SAND.value 
            break
        elif (g is not None):
            if g[0] == 0:
                field = np.pad(field, ((0,0), (1,0)))
                field[h] = Field.ROCK.value
                SEED[0] += 1
            elif g[0] == (field.shape[1] - 1):
                field = np.pad(field, ((0,0), (0,1)))
                field[h] = Field.ROCK.value

    return sum((field == Field.SAND.value).flatten())


In [2]:
field = parseField("test_input.txt")

result, expected = solvePart1(deepcopy(field)), 24
assert result == expected, f"Part 1: {result=} is wrong ({expected=})"

result, expected = solvePart2(deepcopy(field)), 93
assert result == expected, f"Part 2: {result=} is wrong ({expected=})"


In [3]:
field = parseField("input.txt")

print(solvePart1(deepcopy(field)))
print(solvePart2(deepcopy(field)))

838
27539
